<a href="https://colab.research.google.com/github/Hardik357/MNIST-GANs/blob/main/GAN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import time

In [38]:
(train_images, train_labels), (_, _) = keras.datasets.fashion_mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images-127.5)/127.5

In [39]:
BUFFER_SIZE=train_images.shape[0]
BATCH_SIZE = 100
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [40]:
def make_discriminator_model():
    model = keras.Sequential()
    model.add(layers.Conv2D(7, (3,3), padding='same', input_shape=(28,28,1)))
    model.add(layers.Flatten())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(1))
    return model

In [41]:
model_discriminator = make_discriminator_model()
model_discriminator(np.random.rand(1, 28,28,1).astype('float32'))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.05116262]], dtype=float32)>

In [42]:
discriminator_optimizer = tf.optimizers.Adam(1e-3)

In [43]:
def get_discriminator_loss(real_predictions, fake_predictions):
    real_predictions = tf.sigmoid(real_predictions)
    fake_predictions = tf.sigmoid(fake_predictions)

    real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_predictions), real_predictions)
    fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_predictions), fake_predictions)
    return fake_loss + real_loss

In [44]:
def make_generator_model():
    model = keras.Sequential()
    model.add(layers.Dense(7*7*256, input_shape=(100, )))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((7,7,256)))

    model.add(layers.Conv2DTranspose(128, (3,3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))
    return model

In [45]:
generator = make_generator_model()

In [46]:
def get_generator_loss(fake_predictions):
    fake_predictions = tf.sigmoid(fake_predictions)
    fake_loss = tf.losses.binary_crossentropy(tf.ones_like(fake_predictions), fake_predictions)
    return fake_loss

In [47]:
generator_optimizer = tf.optimizers.Adam(1e-4)

In [51]:
def train(dataset, epochs):
    for _ in range(epochs):
        for images in dataset:
            images = tf.cast(images, tf.dtypes.float32)
            train_step(images)

In [49]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100]).astype('float32')
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise)
        real_output = model_discriminator(images)
        fake_output = model_discriminator(generated_images)

        gen_loss = get_generator_loss(fake_output)
        disc_loss = get_discriminator_loss(real_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminant = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminant, model_discriminator.trainable_variables))

        print('generator loss: ', np.mean(gen_loss))
        print('discriminator loss: ', np.mean(disc_loss))

In [52]:
train(train_dataset, 2)

AttributeError: ignored